In [1]:
import pandas as pd
from shapely import Polygon
import geopandas as gpd

In [2]:
parent_filepath = "../../../../../../Simulated WS Sampling/Test Data"
sampleid = "13_G"
analysis_region = "Partition Zone"
cell_name = "CD8"

In [3]:
filepath = f"{parent_filepath}/{sampleid}"

In [4]:
object_data = pd.read_csv(f"{filepath}/Object_Data.csv")

In [5]:
object_data = object_data[["Analysis Region", "Algorithm Name", "XMin", "XMax", "YMin", "YMax", f"{cell_name}"]]

In [6]:
object_data = object_data[(object_data["Analysis Region"] == "Partition Zone") & (object_data["CD8"] == 1)]

In [7]:
cell_coords = []
for idx, row in object_data.iterrows():
    x = (row["XMin"] + row["XMax"]) / 2
    y = (row["YMin"] + row["YMax"]) / 2
    cell_coords.append((x, y))

object_data.loc[:, "Cell Centre"] = cell_coords

object_data = object_data.drop(["XMin", "XMax", "YMin", "YMax"], axis=1)

object_data = object_data.reset_index(drop=True)

In [8]:
object_data

,Analysis Region,Algorithm Name,CD8,Cell Centre
0,Partition Zone,MPIHC_MF_v14_I,1.0,"(56634.5, 16436.0)"
1,Partition Zone,MPIHC_MF_v14_I,1.0,"(57111.5, 16525.0)"
2,Partition Zone,MPIHC_MF_v14_I,1.0,"(56992.5, 16621.0)"
3,Partition Zone,MPIHC_MF_v14_I,1.0,"(57710.5, 16669.5)"
4,Partition Zone,MPIHC_MF_v14_I,1.0,"(58253.0, 16359.5)"
...,...,...,...,...
54366,Partition Zone,MPIHC_MF_v14_I,1.0,"(78374.5, 88053.0)"
54367,Partition Zone,MPIHC_MF_v14_I,1.0,"(78247.0, 88351.5)"
54368,Partition Zone,MPIHC_MF_v14_I,1.0,"(77330.0, 88763.5)"
54369,Partition Zone,MPIHC_MF_v14_I,1.0,"(77290.0, 88761.0)"


In [9]:
with open(f"{filepath}/Annotations.annotations", "r") as anno_file:
    lines = anno_file.readlines()

In [14]:
positive_region = []
layers = []
polygons = []
vertices = []
layer_name = []
region_status = []
for line in lines:
    if "<Annotation Name=" in line:
        # Start a new polygon, pull name from line
        layer_name = line.split('Name="')[1].split('" Visible=')[0]
    
    elif "<Region Type=" in line:
        # Ascertain if polygon is positive or negative
        region_status = line.split('NegativeROA="')[1].split('">')[0] == '0'
        vertices = []
    
    elif "<V X=" in line:
        # Extract the x, y coords from the line
        x = int(line.split('X="')[1].split('" Y=')[0])
        y = int(line.split('Y="')[1].split('" />')[0])
        vertices.append((x, y))
    
    elif "</Region>" in line:
        # End current polygon and add it to the list
        if len(vertices) >= 4:
            polygons.append(Polygon(vertices))
            layers.append(layer_name)
            positive_region.append(region_status)
        else:
            print(f"Small annotation detected for {sampleid}, excluding annotation with {len(vertices)} vertices.")

In [23]:
annos = pd.DataFrame(
    {"Layer": layers,
     "Positive Region": positive_region,
     "Polygon": polygons,
     "Area": [Polygon(i).area for i in polygons]
     }
     )

layer_filt = ["Tumour", "Peritumoral Zone", "Partition Zone"]
annos = annos[annos["Layer"].isin(layer_filt)]

annos = annos.sort_values(["Layer", "Positive Region", "Area"], ascending=[False, False, False]).reset_index(drop=True)

# Compile relevant annotation layers
annos = annos[
    ((annos["Layer"] == "Partition Zone")
     & (annos["Positive Region"] == 0))
     |
    ((annos["Layer"] == "Tumour")
     & (annos["Positive Region"] == 1))
      ]

annos = annos.reset_index(drop=True)

In [24]:
annos

,Layer,Positive Region,Polygon,Area
0,Tumour,True,"POLYGON ((83363 56818, 83370 56811, 83479 5681...",2.224019e+09
1,Tumour,True,"POLYGON ((93173 29913, 93173 29908, 93168 2985...",3.260916e+06
2,Partition Zone,False,"POLYGON ((74530 60152, 74513 60153, 74463 6015...",1.810877e+07
3,Partition Zone,False,"POLYGON ((71949 45539, 71960 45539, 72089 4553...",7.680434e+06
4,Partition Zone,False,"POLYGON ((62433 64799, 62424 64799, 62227 6479...",3.783865e+06
5,Partition Zone,False,"POLYGON ((64707 56625, 64702 56625, 64702 5663...",3.665151e+06
6,Partition Zone,False,"POLYGON ((47134 48789, 47135 48802, 47072 4891...",2.825684e+06
7,Partition Zone,False,"POLYGON ((82054 30449, 82055 30429, 82158 3036...",1.391197e+06
8,Partition Zone,False,"POLYGON ((87722 58713, 87723 58696, 87706 5861...",5.131520e+05
9,Partition Zone,False,"POLYGON ((93684 48808, 93702 48826, 93787 4887...",4.566695e+05
